<a href="https://colab.research.google.com/github/mychele-larson/Big_Data_ETL/blob/main/Big_Data_Pets_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [75]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-12 17:49:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.6’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.72MB/s    in 0.2s    

2023-03-12 17:49:52 (5.72 MB/s) - ‘postgresql-42.2.9.jar.6’ saved [914037/914037]



In [76]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [77]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [78]:
# Get the number of rows in the DataFrame.
df.count()

2643619

# Transform the Data

## Create the "review_id_table".

In [31]:
from pyspark.sql.functions import to_date, col
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
# Select the necessary columns

review_id_df = df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))

# Cast "customer_id" and "product_parent" column to integer
review_id_df = review_id_df.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))

# Show the resulting DataFrame
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789| 2015-08-31|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483| 2015-08-31|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658| 2015-08-31|
|R33GITXNUF1AD4|   33930128|B00BOEXWFG|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|   43534290|B001HBBQKY|     420905252| 2015-08

## Create the "products" Table

In [79]:
from pyspark.sql.functions import to_date, col, substring
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title" columns. 
products_df = df.select("product_id", substring(col("product_title"), 1,300).alias("product_title"))

# Drop duplicates
products_df = products_df.drop_duplicates()

# Show the resulting DataFrame
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
|B004UUE26O|Animal Planet PET...|
|B008APML2C|Chuckit Medium Ul...|
|B00ZJN7T8E|1 Half Portion Ja...|
|B00NRZC8LY|Hide-A-Toy Hallow...|
|B001P3NU4E|Virbac C.E.T. Enz...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B00DJSNF0M|Multipet Lenny th...|
|B00FXVFEQG|Bags on Board Dur...|
|B005ORDWYA|Just One Bite II ...|
|B003E77OG4|  Petmate Sky Kennel|
+----------+--------------------+
only showing top 20 rows



In [80]:
# Group by columns to find duplicates
duplicates = products_df.groupBy("product_id", "product_title").count().filter(col("count") > 1)

# Show duplicates
duplicates.show()

+----------+-------------+-----+
|product_id|product_title|count|
+----------+-------------+-----+
+----------+-------------+-----+



## Create the "customers" Table

In [33]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))

# Show the resulting DataFrame
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30238476|             1|
|   51090028|             1|
|   14286306|            10|
|    3223564|             1|
|   16794688|             5|
|   51959139|             2|
|   24568606|             1|
|   15205394|             5|
|   44333804|             2|
|   33982951|             3|
|   49728087|             1|
|     169338|             1|
|   50583551|            11|
|   47379266|             1|
|   48428870|            26|
|   10250915|            10|
|   39661665|             2|
|   27199526|             1|
|   19685782|             9|
|   46783450|             3|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [34]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))
            
# Show the resulting DataFrame
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
|R33GITXNUF1AD4|          2|            0|          0|   N|
|R1H7AVM81TAYRV|          1|            2|          2|   N|
|R2ZOYAQZNNZZWV|          5|            0|          0|   N|
|R2FN1H3CGW6J8H|          1|            

# Load Postgres Set-up

In [81]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://rds-database.cjffwh6trilr.us-east-1.rds.amazonaws.com:5432/BigData_RDS"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [38]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [82]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [39]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [40]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)